In [ ]:
from pandas import DataFrame
from astropy.time import Time, TimeDelta
from dateutil import parser
from IPython.display import display, Markdown

In [ ]:
log_date_param = '2022-04-05'
# need to account for daylight savings
log_date = Time(parser.isoparse(f'{log_date_param}T00:00:00.00-04:00'))

In [ ]:
data = {}
data['Observation Time*'] = [10.2,]
data['Science'] = [6.6,]
data['Weather Loss'] = [0.,]
data['Technical faults'] = [1.3,]
data['Software faults'] = [0.4,]
data['Engineering'] =[1.6,]
data['Commissioning'] = [0.,]
data['Education**'] = [0.3,]
time_accounting = DataFrame.from_dict(data)
time_accounting.to_csv('time_accounting.csv')

In [ ]:
ops_status = \
'''
* &#11096; Any large hexapod motions must happen near the zenith to avoid surpassing the maximum load permitted by the hexapod.
* &#11096; Fan should not run above 50% capacity when being used on-sky.
* &#11096; **Doing snaps this run** – see details in https://sitcomtn-032.lsst.io/ 
* &#11096; Scripts now use the "time_on_target" parameter to optimize the azimuth and rotator position
    * &#9654; time estimate is primarily based on exposure time and estimated overheads
        * &#10141; If you hit a limit please let Patrick know!
* &#11096; Mount motion issues (unchanged)
    * &#9654; Settling
        * &#10141; Now have the a parameters to adjust the mount settling (this is not going to help the blips seen at the pole)
            * &bull; Tiago has removed the ATCS bug, now relying fully on the ATMCS
        * &#10141; Please flag where exposures are bad after a slew/offset
    * &#9654; Tracking 
        * &#10141; A very slow speeds (the pole) - this issue remains
        * &#10141; At high speeds (cogging) - you probably won't actually see this visually, but it will slightly degrade the PSF
    * &#9654; Nasymth over-torque:
        * &#10141; Need more data to track down where/when this occurs, just log when it faults and continue
    * &#9654; Others (including wind)
        * &#10141; Take accelerometer data!
        * &#10141; Pause scriptQueue, take data with accelerometers, then resume the scriptQueue 
'''
with open('ops_status.txt', 'w') as fh:
    fh.write(ops_status)
display(Markdown(ops_status))

In [ ]:
night_plan = 'Scheduler-driven imaging and engineering tasks.'
with open('night_plan.txt', 'w') as fh:
    fh.write(night_plan)
display(Markdown(night_plan))

In [ ]:
night_summary = 'A bit of a rocky start with the k8s cluster going down and other errors, but we accomplished a lot of engineering tasks and completed the imaging survey, including repeat visits.'
with open('night_summary.txt', 'w') as fh:
    fh.write(night_summary)
display(Markdown(night_summary))

In [ ]:
def make_pretty(styler):
    styler.format(lambda x: f'{x:.2f}' if (isinstance(x, float) or isinstance(x, int)) else x)
    styler.hide(axis='index')
    styler.background_gradient(axis=None, vmin=0, vmax=12., cmap='YlGnBu')
    return styler

display(time_accounting.style.pipe(make_pretty))

In [ ]:
time_accounting.to_csv('time_accounting.csv')

In [ ]:
def make_pretty_times(styler):
    styler.format(lambda x: f'{x:.2f}' if (isinstance(x, float) or isinstance(x, int)) else x)
    styler.background_gradient(axis=None, vmin=0, vmax=6., cmap='YlGnBu')
    styler.format_index(lambda v: v.strftime("UTC %c"))
    return styler

def make_times(dts, btime):
    times = []
    for dt in dts:
        secs = sum([a*b for a, b in zip(sec_arr, [int(el) for el in dt.split(':')])])
        if secs < 12*3600:
            secs += 24*3600
        times.append(btime + TimeDelta(secs, format='sec'))
    return times

data = {}

obs_times = ['19:30', '20:00', '20:30', '20:45', '23:25', '3:16:42']
data['Time (summit)'] = make_times(obs_times, log_date)
data['Summary'] = ['Dropped LATISS image', 'DIMM failed to startup',
                   'DIMM CSC publishing fake data', 'notebook server/k8s cluster crash and reboot',
                   'Nasmyth 2 max velocity exceeded', 'Test entry for past midnight entries']
data['Ticket number'] = ['OBS-25', 'OBS-27', 'OBS-26', '--', 'DM-30709', '--']
data['Sky time loss/Time to recover (hours)'] = [0.0, 0.0, 0.0, 0.6, 0.7, 5.0]
data['Notes'] = ['', '', '', '', 'fault loss due to recovering the pointing after moving nasymith 2 with EUI',
                 'loss due to Simon falling asleep at the wheel']

fault_reports = DataFrame.from_dict(data)
fault_reports = fault_reports.set_index('Time (summit)')
fault_reports.to_csv('faults.csv')
display(fault_reports.style.pipe(make_pretty_times))


In [ ]:
data = {}

obs_times = ['20:00', '23:00', '02:00']
data['Time (summit)'] = make_times(obs_times, log_date)
data['Conditions'] = ['clear, calm', 'clear/calm', 'clear']
data['Humidity'] = [22.5, 19, 18]
data['Wind speed (m/s)'] = [2, 3.5, 5.1]
data['Direction (degrees)'] = [303, 319, 303]
data['DIMM seeing'] = ['(Gemini) 0.52"', 'Rubin 1.44"', 'Rubin 1.13"']

weather = DataFrame.from_dict(data)
weather = weather.set_index('Time (summit)')
weather.to_csv('weather.csv')
display(weather.style.pipe(make_pretty_times))

In [ ]:
data = {}

data['Tasks'] = ['Practice of scheduler operations on the TTS',
                 'Train operators to run/read Craig\'s accelerometer notebook(s?)',
                 'Test "time on target" parameter of slew_target',
                 'Update image type for acquisitions in scripts']
data['Notes'] = ['',
                 '',
                 'TTS unavailable, will need to verify on-sky',
                 'Consider either modifying the scheduler plan to stay on target without continually re-acquiring, or find a new way to check whether or not we\'re already acquired via the script+Scheduler']

day_tasks = DataFrame.from_dict(data)
day_tasks.to_csv('day_tasks.csv')
display(day_tasks.style.pipe(make_pretty))

In [ ]:
data = {}

obs_times = ['15:30', '16:23', '18:03', '19:30', '19:35', '19:42', '19:52', '19:54', '20:00', '20:30', '20:43', '20:51', '21:00', '21:11',
             '21:18', '21:20', '21:24', '21:39', '21:43', '21:44', '21:49', '21:56', '22:19', '23:25', '23:28', '00:02', '00:05', '00:10',
             '00:20', '00:25', '00:38', '00:40', '00:45', '00:50', '00:55', '02:21', '04:35', '04:41', '04:43', '06:02', '06:04', '06:10']
data['Time (summit)'] = make_times(obs_times, log_date)
data['Event'] = ['Night crew arrives to summit, beginning daily checkout', 'Beginning afternoon calibrations.',
                 'Running prepare for onsky with script queue', 'Attempted to run scriptQueue wfs script, but it failed',
                 'Initiated script again, completed successfully.', 'Determine that this failed due to a dropped image 207 - no butler image',
                 'Running acquire and take sequence to absorb pointing offsets', '12 degree twilight, beginning of science time',
                 'Running scheduler driven operations', 'Relying on Gemini DIMM data for the moment, in the process of troubleshooting Rubin DIMM - See OBS-27',
                 'Scheduler went into fault - beginning technical time loss', 'Unable to talk to EFD, and notebook servers have crashed',
                 '*Lost all CSCs on the k8 clusters*', '*After cluster was re-started, recovered ATCS and CSCs*', 'Resumed scheduler operations, starting with cwfs - end technical time loss',
                 'DIMM begins working after some trouble.', 'We did not enable ataos corrections, scheduler went into fault - educational time', 'Running a new focus script separate from scheduler - establishing new baseline',
                 'Script failed due to a second bright star in the field - ts_external_scripts was on the wrong branch',
                 'Re-running script on a new target - end of educational time', 'Resuming scheduler-driven operations',
                 'Interrupt scheduler-driven observations to go for engineering', 'Resume scheduler driving operations after environment troubles',
                 'ATPtg went into fault, nasmyth 2 max velocity error exceeded - technical fault', 'Recovered', 'Nasmyth 2 max velocity exceeded, causing a ATPtg fault. Recovered CSCs.',
                 'Going to rotate nasmyth with the EUI from 132 → 20 to try and "kick the error out." Be gone, Beezulbub!',
                 'After going in through the EUI and trying to force the nasmyth, the telescope does not respond to slew/pointing commands.',
                 'Sending ATPTg and ATMCS CSCs to standby and back to enabled.', 'Telescope not receiving target commands.', 'Sending the pointing to offline, and then back online. Able to slew.',
                 'Beginning some engineering tasks.', 'Eng#1_Donut_Pairs Eng#1_Donut_Pairs.pdf', 'Eng#3_Parabolic Focus Sweep  Eng#3_Parabolic_Focus_Sequence.pdf',
                 'Eng#2_PSF_Ellipticity Eng#2_PSF_Ellipticipy.pdf', 'Going to go back to scheduler.', 'Notice that scriptQueue is hung up on wfs calculation, possibly due to another bright target in the field.',
                 'Kill the step, continue with imaging.', 'Load a separate cwfs to catch a focus and continue with survey.',
                 'Scheduler goes into fault due to not having another target - calling the night.', 'Beginning of nautical twilight. Running shutdown script.',
                 'Night crew departing summit.']
data['Target'] = ['' for el in data['Time (summit)']]
data['Exposure numbers(s)'] = ['' for el in data['Time (summit)']]
data['Exposure Time (secs)'] = ['' for el in data['Time (summit)']]
data['Exposure Time (secs)'][32] = '(449 to 521)'
data['Exposure Time (secs)'][33] = '(523, 531)'
data['Exposure Time (secs)'][34] = '(536, 601) are these actually expsure numbers?'
data['Good Spectrum?'] = ['' for el in data['Time (summit)']]
data['Comments'] = ['' for el in data['Time (summit)']]
data['Comments'][3] = """Error in run

b'Traceback (most recent call last):\n File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-3.0.0/lib/python3.8/site-packages/lsst/ts/salobj/base_script.py", line 589, in do_run\n await self._run_task\n File "/home/saluser/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_cwfs_align.py", line 974, in run\n await self.arun(True)\n File "/home/saluser/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_cwfs_align.py", line 869, in arun\n results = await self.run_cwfs()\n File "/home/saluser/ts_externalscripts/python/lsst/ts/externalscripts/auxtel/latiss_cwfs_align.py", line 389, in run_cwfs\n self.intra_exposure, self.extra_exposure = await asyncio.gather(\n File "/home/saluser/ts_observing_utilities/python/lsst/ts/observing/utilities/auxtel/latiss/getters.py", line 66, in get_image\n raise TimeoutError(\nTimeoutError: Unable to get raw image from butler in 20.0 seconds.'

See OBS-25"""
data['Comments'][10] = """Error on advance target production loop.
Traceback (most recent call last): File "/opt/lsst/software/stack/miniconda/lib/python3.8/site-packages/aiohttp/connector.py", line 1154, in _create_direct_connection hosts = await  aiohttp.client_exceptions.ClientConnectorError: Cannot connect to host influxdb-summit-efd.lsst.codes:30828 ssl:default [Name or service not known]"""
data['Comments'][23] = 'DM-30709'
data['Comments'][29] = 'Exception image'
data['Comments'][34] = 'Link to sequence list'
night_log = DataFrame.from_dict(data)
night_log = night_log.set_index('Time (summit)')
night_log.to_csv('night_log.csv')
display(night_log.style.pipe(make_pretty_times))

In [ ]:
print(night_log)